In [29]:
%matplotlib widget
import numpy as np
np.random.seed(69420)
import seaborn as sns
import pandas as pd
from scipy import spatial, stats
from functools import reduce
import matplotlib.pyplot as plt
import nrrd
import pytest
import ipytest
ipytest.autoconfig(raise_on_error=True)
import raster_geometry as rg
import sys
import os
import cv2
sys.path.append("C:\\Users\\danie\\BoneSegmentation")

from oiffile import OifFile
from src.OifImageViewer import OifImageViewer

In [ ]:
def euclidean_distance(point1, point2):
    return np.linalg.norm(point1-point2)

In [35]:
test_random_points = stats.uniform.rvs(0, 100, (100,3))
print(test_random_points)
print(test_random_points.dtype)
# N = random_points.shape[0]


[[58.43626324 42.90627363 38.11072369]
 [34.8774914  73.95664544 24.16186084]
 [58.84139909 69.25546845 77.14984347]
 [68.79536612 99.63332347  5.7616175 ]
 [49.14133503 15.17926622  8.66968748]
 [66.73111611 82.34631025 96.91848343]
 [77.48450271 19.50926592  1.27354382]
 [ 5.65666706 22.42845248  8.1219263 ]
 [72.29416506 33.84284775 26.50350931]
 [79.95331306 59.91790798 84.59454779]
 [52.45018483 88.63591365  0.82948286]
 [58.3127736  95.73596091 85.38521188]
 [93.48564439 77.93281055  1.17483959]
 [ 9.85682166 12.67796324  9.22425097]
 [22.65834259 97.47819199 28.88442821]
 [29.77163283 73.79782624 49.14025788]
 [63.47120852 67.72181758 21.55378121]
 [21.10250873  6.25149988 24.11413817]
 [32.37823423 78.90054922 91.02676895]
 [37.78003295  5.5003913  66.42468707]
 [91.95917864 91.80034834 78.91011508]
 [68.11596044 99.37740846 65.78093434]
 [67.78599527  7.33089237  7.84822294]
 [65.55207321 29.58541    24.45313541]
 [86.85157153 86.24584488 98.99538707]
 [69.3826597  72.42091963

In [ ]:
# Test distance function
print(euclidean_distance(np.array((10,10,10)),np.array((0,0,0))))

In [ ]:
def set_aspect_ratio_equal(ax) -> None:
    xlim = ax.get_xlim3d()
    ylim = ax.get_ylim3d()
    zlim = ax.get_zlim3d()
    ax.set_box_aspect((xlim[1]-xlim[0], ylim[1]-ylim[0], zlim[1]-zlim[0]))

In [ ]:
def calculate_weight(shape, radius, midpoint):
    ### Calculate proportion of sphere within study volume
    # check that radius is smaller than half of the smallest dimension
    assert radius <= min(shape) / 2
    assert radius > 0

    # determine center of study volume
    center = (shape[0]//2, shape[1]//2, shape[2]//2)

    # draw target sphere in numpy space at specified midpoint and reference sphere at middle of volume
    target = rg.nd_superellipsoid(shape, radius, position=midpoint, rel_sizes=False, rel_position=False)
    reference = rg.nd_superellipsoid(shape, radius, position=center, rel_sizes=False, rel_position=False)

    # calculate proportion of volumes to determine weight
    target = target.astype(np.int_).sum()
    reference = reference.astype(np.int_).sum()
    print(target, reference)
    return target / reference

In [ ]:
%prun calculate_weight((100,100,100), 7, (2,2,2))

In [ ]:
# Test sphere drawing
def draw_sphere(shape, radius, center) -> np.ndarray:
    sphere = rg.nd_superellipsoid(shape, radius, position=center, rel_sizes=False, rel_position=False)
    return sphere.astype(np.int_)

container_shape = (100, 100, 100)
radius = 7
center = (2, 2, 2)

# print(calculate_weight(container_shape, radius, center))
test_sphere = draw_sphere(container_shape, radius, center)

from mpl_toolkits.mplot3d import Axes3D
z,x,y = test_sphere.nonzero()
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
set_aspect_ratio_equal(ax)
ax.axes.set_xlim3d(left=0, right=container_shape[1]) 
ax.axes.set_ylim3d(bottom=0, top=container_shape[2]) 
ax.axes.set_zlim3d(bottom=0, top=container_shape[0]) 
ax.scatter(x, y, z, zdir='z', c= 'red')
plt.show()

In [ ]:
x_min = 0; x_max = 100
y_min = 0; y_max = 100
z_min = 0; z_max = 100
total_volume = (x_max - x_min)*(y_max - y_min)*(z_max - z_min)

In [ ]:
# Currently using x_max-x_min as maximum radius, by convention it should be sqrt(x_max-x_min/2)
radii = np.linspace(0,x_max-x_min,50).reshape(50,1)

In [ ]:
test_tree = spatial.cKDTree(test_random_points)
idx_in = test_tree.query_ball_point((50,50,50),20)

idx_out = np.ones(len(test_random_points), bool)
idx_out[idx_in] = 0

pts_inside_radius = test_random_points[idx_in]
pts_outside_radius = test_random_points[idx_out]

print(len(test_random_points), len(pts_inside_radius), len(pts_outside_radius))

In [ ]:
ix, ox = pts_inside_radius[:, 0], pts_outside_radius[:, 0]
iy, oy = pts_inside_radius[:, 1], pts_outside_radius[:, 1]
iz, oz = pts_inside_radius[:, 2], pts_outside_radius[:, 2]

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
set_aspect_ratio_equal(ax)
ax.scatter(ix,iy,iz, color="b",marker="o", depthshade=0)
ax.scatter(ox,oy,oz,color="r",marker="o", depthshade=1)


scale = 20
# draw sphere
u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
cx = scale*np.cos(u)*np.sin(v)+50
cy = scale*np.sin(u)*np.sin(v)+50
cz = scale*np.cos(v)+50
ax.plot_wireframe(cx, cy, cz, color="gray", alpha=0.3)


In [ ]:
# For this rectangular test case, volume_mask is just an array of ones
volume = 100*100*100
volume_mask = np.ones((100, 100, 100))
volume_shape = volume_mask.shape

print(volume)
print(volume_mask.sum())

Ripley's K Estimators

![equation](http://www.sciweavers.org/tex2img.php?eq=%5Cwidehat%7BK%7D%28r%29%3D%20%5Cfrac%7BV%7D%7BN%28N-1%29%7D%20%5Csum_i%5EN%20%5Csum_%7Bj%20%5Cneq%20i%7D%5EN%20%20%5Cfrac%7BI%28d_%7Bi%2Cj%7D%3Cr%29%7D%7B%5Comega_%7Bi%2Cj%7D%7D%20%20%20%20%20&bc=White&fc=Black&im=jpg&fs=18&ff=arev&edit=0)

![equation](http://www.sciweavers.org/tex2img.php?eq=%5Cwidehat%7BL%7D%28r%29%3D%20%20%20%5Csqrt%5B3%5D%7B%5Cfrac%7B3%20%20%5Cwidehat%7BK%7D%28r%29%7D%7B4%5Cpi%7D%7D&bc=White&fc=Black&im=jpg&fs=18&ff=arev&edit=0)

![equation](http://www.sciweavers.org/tex2img.php?eq=%5Cwidehat%7BH%7D%28r%29%3D%5Cwidehat%7BL%7D%28r%29-r&bc=White&fc=Black&im=jpg&fs=12&ff=arev&edit=0)

In [ ]:
def ripley(points: np.ndarray, radii: list, volume_shape: tuple, boundary_correction: bool = True) -> tuple:
    """
    Calculate RipleyK spatial statistic on 3D point cloud

    args:
        points (numpy.ndarray): shape (N, 3), where N is the number of points
        radii (list[int]): list of radii to compute
        volume_shape (tuple[int]): shape of the entire study volume. Example: (100,100,100)

    """
    ### Function starts with input checks

    # Check if points is a list or numpy array
    if not isinstance(points, (list, np.ndarray)):
        e = f"Expected {np.ndarray}, received {type(points)}"
        raise ValueError(e)

    # Convert points to numpy array if it is a list
    if not isinstance(points, np.ndarray):
        points = np.array(points)

    # Check if points array has two dimensions
    if len(points.shape) != 2:
        e = f"Expected points array to have 2 dimensions, but got array with shape {points.shape}"
        raise ValueError(e)

    # Check if the points array second dimension length is 3 (x, y, z)
    if points.shape[1] != 3:
        e = f"Expected points array to have shape (None, 3), but got array with shape {points.shape}"
        raise ValueError(e)

    # Check if the points array has at least 3 points
    if points.shape[0] < 3:
        e = f"Expected points array to have at least 3 points"
        raise ValueError(e)

    # Check if radii is list-like or number-like
    if not isinstance(radii, (np.ndarray, list, int, float)):
        e = f"Expected {(np.ndarray, list, int, float)}, received {type(radii)}"
        raise ValueError(e)

    # if only one radius given as int, convert to list
    if not isinstance(radii, (np.ndarray, list)):
        radii = [radii]

    ### Compute Ripley's K Statistic 
    
    results = {"K": [], "L": [], "H": []}
    tree = spatial.cKDTree(points)
    N = points.shape[0]
    study_volume = reduce(lambda x, y: x * y, volume_shape)

    # Loop through each radius
    for radius in radii:
        # For each radius, loop through each point and count points
        # within the radius
        nb_count = 0
        for x, y, z in points:
            # TODO: doublecheck that the z,x,y ordering is correct and properly aligns with mask
            if boundary_correction:
                weight = calculate_weight(volume_shape, radius, (z, x, y))
            else:
                weight = 1.0
            # query_ball_point() includes the index of the current point as well
            # so 1 is subtracted from the count
            nb_count += (len(tree.query_ball_point([x, y, z], radius)) - 1) / weight

        # calculating 3D Ripley's functions (K, L, H)
        K = nb_count * study_volume / (N * (N - 1))
        L = ((3. / 4) * (K / np.pi)) ** (1. / 3)
        H = L - radius

        # Verify K/L values positive
        if K < 0 or L < 0:
            raise ValueError(f"K/L values should not be negative. nb_count: {nb_count}, volume: {volume}, N: {N}")

        results["K"].append(K)
        results["L"].append(L)
        results["H"].append(H)
    
    return results["K"], results["L"], results["H"]




In [ ]:
a = volume_mask.shape




### Complete Spatial Randomness (CSR) Representations

In [ ]:
CSR_data = []
CSR_data_w = []
radii=np.arange(1, 51)
for i in range(5):
    print(i)
    CSR_points = stats.uniform.rvs(0, 100, (100,3))
    K, L, H = ripley(CSR_points, radii, volume_shape, boundary_correction=False)
    K_w, L_w, H_w = ripley(CSR_points, radii, volume_shape, boundary_correction=True)
    CSR_data += zip(radii, K, L, H, ["CSR"]*len(radii), [i]*len(radii))
    CSR_data_w += zip(radii, K_w, L_w, H_w, ["CSR"]*len(radii), [i]*len(radii))
rstats = pd.DataFrame(CSR_data, columns=["Radius (r)", "K(r)", "L(r)", "H(r)", "Type", "Line"])
rstats_w = pd.DataFrame(CSR_data_w, columns=["Radius (r)", "K(r)", "L(r)", "H(r)", "Type", "Line"])



In [ ]:
f, axes = plt.subplots(2, 3,)

## Top Row
# K
sns.lineplot(data=rstats, x="Radius (r)", y="K(r)", ax=axes[0, 0], hue="Line")
# L
sns.lineplot(data=rstats, x="Radius (r)", y="L(r)", ax=axes[0, 1], hue="Line")
axes[0, 1].plot([0, 50], [0, 50], linewidth=1, linestyle="dotted", color="#aaa") # Expected line
# H
sns.lineplot(data=rstats, x="Radius (r)", y="H(r)", ax=axes[0, 2], hue="Line")
axes[0, 2].plot([0, 50], [0, 0], linewidth=1, linestyle="dotted", color="#aaa") # Expected line

## Bottom Row
# K
sns.lineplot(data=rstats, x="Radius (r)", y="K(r)", ax=axes[1, 0], alpha=0, errorbar=("pi", 95))
# L
sns.lineplot(data=rstats, x="Radius (r)", y="L(r)", ax=axes[1, 1], alpha=0, errorbar=("pi", 95))
axes[1, 1].plot([0, 50], [0, 50], linewidth=1, linestyle="dotted", color="#aaa") # Expected line
# H
sns.lineplot(data=rstats, x="Radius (r)", y="H(r)", ax=axes[1, 2], alpha=0, errorbar=("pi", 95))
axes[1, 2].plot([0, 50], [0, 0], linewidth=1, linestyle="dotted", color="#aaa") # Expected line

plt.show()



In [ ]:
f, axes = plt.subplots(2, 3,)

## Top Row
# K
sns.lineplot(data=rstats_w, x="Radius (r)", y="K(r)", ax=axes[0, 0], hue="Line")
# L
sns.lineplot(data=rstats_w, x="Radius (r)", y="L(r)", ax=axes[0, 1], hue="Line")
axes[0, 1].plot([0, 50], [0, 50], linewidth=1, linestyle="dotted", color="#aaa") # Expected line
# H
sns.lineplot(data=rstats_w, x="Radius (r)", y="H(r)", ax=axes[0, 2], hue="Line")
axes[0, 2].plot([0, 50], [0, 0], linewidth=1, linestyle="dotted", color="#aaa") # Expected line

## Bottom Row
# K
sns.lineplot(data=rstats_w, x="Radius (r)", y="K(r)", ax=axes[1, 0], alpha=0, errorbar=("pi", 95))
# L
sns.lineplot(data=rstats_w, x="Radius (r)", y="L(r)", ax=axes[1, 1], alpha=0, errorbar=("pi", 95))
axes[1, 1].plot([0, 50], [0, 50], linewidth=1, linestyle="dotted", color="#aaa") # Expected line
# H
sns.lineplot(data=rstats_w, x="Radius (r)", y="H(r)", ax=axes[1, 2], alpha=0, errorbar=("pi", 95))
axes[1, 2].plot([0, 50], [0, 0], linewidth=1, linestyle="dotted", color="#aaa") # Expected line

plt.show()

In [ ]:
%%ipytest

# Tests
def test_valid_input():
    valid_points = [
        np.array([[38.4002419, 38.62931512, 41.6131775], [38, 38, 41], [1, 6, 8]]),
        np.array([[38, 38, 41], [1, 2, 3], [20, 20, 20]]),
        [[38.4002419, 38.62931512, 41.6131775], [3.4002419, 8.62931512, 1.6131775], [8.4002419, 3.62931512, 4.6131775]],
        [[38, 38, 41], [1, 2, 3], [4, 5, 6]],
    ]
    for p in valid_points:
        ripley(p, radii=1, volume=100*100*100)

def test_invalid_input():
    invalid_points = [
        np.array([[38.4002419, 38.62931512, 41.6131775]]),
        np.array([[38, 38, 41], [1, 2, 3]]),
        [[38.4002419, 38.62931512, 41.6131775]],
        [[38, 38, 41], [1, 2, 3]],
        "38.4002419, 38.62931512, 41.6131775",
        np.array([38.4002419, 38.62931512, 41.6131775]),
        np.array([[[38.4002419, 38.62931512, 41.6131775]]]),
        np.array([[38.4002419, 38.62931512]]),
        5,
        7.8,
        [6],
    ]
    for p in invalid_points:
        with pytest.raises(ValueError):
            ripley(p, radii=1, volume=100*100*100)


In [30]:
# path = "/home/dkermany/data/Bone_Project/weijie_selected/main_folder/FV10__oibs/FV10__20190620_111343.oib"
path = "D:\\Datasets\\Bone_Mets\\FV10__oibs\\FV10__20190620_111343.oib"
filename = os.path.splitext(os.path.basename(path))[0]

with OifFile(path) as oif:
    viewer = OifImageViewer(oif)
    x_step, y_step, z_step = map(float, (viewer.md["x_step"], viewer.md["y_step"], viewer.md["z_step"]))
    if viewer.md["z_unit"] == "nm":
        z_step /= 1000.

Image shape: (3, 76, 2404, 3376)
Axes: CZYX
Dtype: uint16
Intervals: X (1.242um) Y (1.242um) Z (2660nm)



In [ ]:
mask_path = "D:\\Datasets\\Bone_Mets\\masks\\FV10__20190620_111343.seg.nrrd"

# Load mask from NRRD
mask, header = nrrd.read(mask_path)
mask = mask.T
mask = mask[:mask.shape[0]//3, :, :]

print(mask.shape)
print(mask.dtype)



In [ ]:
tumor_csv_path = "D:\\Datasets\\Bone_Mets\\tumor_locations.csv"
# Load csv and drop rows with N/A's
tumor_csv = pd.read_csv(tumor_csv_path).dropna()
tumor_csv

In [ ]:
tumor_dict = tumor_csv.to_dict("index")
tumor_dict

In [31]:
import math
tumor_labels = np.zeros(mask.shape, dtype=np.uint16)
for idx, tumor_info in tumor_dict.items():
    x_um, y_um, z_slice = map(int, (tumor_info["x (um)"], tumor_info["y (um)"], tumor_info["z (slice)"]))
    x, y, z = map(int, (x_um / x_step, y_um / y_step, z_slice-1))
    print(x, y, z)
    
    sphere_radius = 7 # micron
    sphere_z_slices = math.ceil(sphere_radius / z_step)
    
    min_z_step = max(0, z-sphere_z_slices)
    max_z_step = min(z+sphere_z_slices, viewer._arr.shape[1])

    # Label tumor
    for z_prime in range(min_z_step, max_z_step):
        cv2.circle(
            tumor_labels[z_prime],
            (x, y),
            radius=round((sphere_radius**2 - min((z_step*abs(z_prime-z))**2, sphere_radius**2))**0.5),
            color=(4096,4096,4096),
            thickness=-1,
        )
print(tumor_labels.shape)

2404 1417 75
2395 1413 75
2377 1380 75
2479 1352 75
2266 1080 75
730 989 75
2427 704 75
2432 693 75
2459 732 75
2633 840 75
2675 600 75
2771 595 75
2315 1432 66
2384 710 64
2351 721 62
2518 747 61
2344 1119 59
2301 1399 58
1892 647 53
1892 647 53
2336 1417 51
2351 1423 51
2799 592 48
2367 1393 43
1942 685 42
2411 1130 36
1928 668 35
2030 1294 26
1731 1243 26
823 752 13
1690 1226 4
2243 1351 0
2159 1356 0
2159 1338 0
2531 1475 0
1386 448 74
1861 409 73
1409 425 28
1202 254 40
1148 658 14
1095 920 33
1067 866 53
659 1099 40
764 1188 46
1697 847 12
1782 722 11
1772 324 11
2214 775 8
1861 409 73
1835 388 51
1834 418 45
1810 479 44
1615 343 43
1843 348 25
2213 254 21
2206 276 22
2044 756 12
1967 801 9
2002 742 33
1965 752 38
2413 669 50
2294 685 61
2332 412 73
2309 467 67
2595 423 67
2219 384 61
2202 298 60
2296 338 55
2182 313 48
2456 440 48
2302 451 49
2331 440 44
2594 473 40
2534 674 72
3064 362 34
3083 347 68
3191 650 35
3003 236 29
(76, 2404, 3376)


In [ ]:
def calculate_ripley(radii, sample_size, d1=None, d2=None, d3=None, sample_shape='circle', boundary_correct=False, CSR_Normalise=False):
    results = []
    tree, dimensions = make_tree(d1=d1, d2=d2, d3=d3)
    if type(radii) is not list:
        radii = [radii]
    for radius in radii:
        
        score_vol = (4/3) * np.pi * radius**3
        
        if sample_shape=='circle':
            bound_size = (4/3) * np.pi * sample_size ** 3
        elif sample_shape=='rectangle':
            bound_size = sample_size[0]*sample_size[1]*sample_size[2]
        counts = 0
        for x, y, z in zip(d1, d2, d3):
            if boundary_correct:
                vol = calculate_overlap([x,y,z], sample_size, radius, sample_shape, dimensions)
                boundary_correction = vol/score_vol
                counts += (len(tree.query_ball_point([x,y,z], radius))-1)/boundary_correction
            else:
                counts += len(tree.query_ball_point([x,y,z], radius))-1
                    
        if CSR_Normalise:
            results.append((bound_size*counts/len(d1)**2) - score_vol)
        else:
            results.append(bound_size*counts/len(d1)**2)
    if len(results)==1:
        return results[0]
    else:
        return results

In [ ]:
# def draw_sphere(arr: np.ndarray, radius: int, coords: list):
#     x_min, x_max = max(coords[0] - radius, 0), min(coords[0] + radius, arr.shape[0])
#     y_min, y_max = max(coords[1] - radius, 0), min(coords[1] + radius, arr.shape[1])
#     z_min, z_max = max(coords[2] - radius, 0), min(coords[2] + radius, arr.shape[2])
#     arr_overlap = [(max(coords[i] - radius, 0), min(coords[i] + radius, arr.shape[i])) for i in range(len(coords))]

#     kernel_size = (2 * radius) + 1
#     sphere_kernel = rg.sphere(kernel_size, r).astype(np.int_)

#     low_indices = [coords[i] - radius if coords[i] - radius > 0 else 0 for i in range(len(coords))]
#     high_indices =

#     idx_low = coords[0] - radius if coords[0] - radius > 0 else 0
#     idx_high = coords[0] + radius if coords[0] + radius >= arr.shape[0] else arr.shape[0]

#     if idx_low < 0:
#         sphere_kernel = sphere_kernel[abs(idx_low):, :, :]
#     if idx_high >= arr.shape[0]:
#         deltaX = arr.shape[0] - idx_high
#         assert deltaX < 0
#         sphere_kernel = sphere_kernel[:deltaX, :, :]


#     return arr[x_min: x_max + 1,
#                y_min: y_max + 1,
#                z_min: z_max + 1]
    
    
# test_vol = np.zeros((10,10,10))
# a = draw_sphere(test_vol, 1, (0,0,0))   
# print(a.shape)
# print(a)